In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

a = ana.Anatomist()

from soma import aims
import pandas as pd
import numpy as np
import os

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module volumepalettes
loading module paletteViewer
loading module modelGraphs
loading module infowindow
loading module profilewindow
loading module bsa_proba
loading module meshsplit
loading module selection
loading module anacontrolmenu
loading module gltf_io
loading module ana_image_math
loading module palettecontrols

loading of module palettecontrols failed:
ImportError : cannot import name 'MiniPaletteGraphics' from 'anatomist.cpp' (/casa/host/build/python/anatomist/cpp/__init__.py)
traceback:
/casa/host/build/python/anatomist/cpp/__init__.py ( 405 ) in __init__ :
six.exec_('import ' + x, {}, {})
<string> ( 1 ) in <module> :

/casa/host/build/share/anatomist-5.2/python_plugins/palettecontrols.py ( 41 ) in <module> :
from anatomist.cpp import palettecontrastaction  # noqa: F401
/casa/host/build/python/anatomist/cpp/palettecontrastaction.py ( 41 ) in <modu

#### To visualize specific crops after a projection

In [2]:
dataset = 'UkBioBank40'
region = "S.Or."
side = "L"

bucket_path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}buckets'
mm_skeleton_path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops'

In [3]:
#sorted_phenotype = pd.read_csv('/volatile/ad279118/Irene/MOSTEST/Results/Imaging_Genetics/25irene_AD_CINGULATE_val/results/ChampollionV0/CINGULATE_left/09-35-58_201/white.British.ancestry/Sorted_projection/regression_on_latent/rs4842267_C_small_distances.csv')
#sorted_phenotype = pd.read_csv('/volatile/ad279118/Irene/MOSTEST/Results/Imaging_Genetics/25irene_AD_CINGULATE_val/results/ChampollionV0/CINGULATE_left/09-35-58_201/white.British.ancestry/Sorted_projection/regression_on_latent/rs2009778_T.csv')
#sorted_phenotype = pd.read_csv('/volatile/ad279118/Irene/MOSTEST/Results/Imaging_Genetics/25irene_AD_CINGULATE_val/results/ChampollionV0/CINGULATE_left/09-35-58_201/white.British.ancestry/Sorted_projection/regression_on_latent/rs12951057_G.csv')
sorted_phenotype = pd.read_csv('/volatile/ad279118/Imaging_Genetics_2025/UKB_cingulate_pred/UKB_left_pred.csv')
sorted_phenotype = sorted_phenotype.sort_values(by='Left_Prob_Pred')
sorted_phenotype.IID = sorted_phenotype['IID'].apply(lambda x : 'sub-'+str(x))
list_subjects = sorted_phenotype['IID'].to_list()
len(list_subjects)
sample = list_subjects[-420:-400]
#sample = ['sub-4157001','sub-4295302','sub-4716793' ]

FileNotFoundError: [Errno 2] No such file or directory: '/volatile/ad279118/Imaging_Genetics_2025/UKB_cingulate_pred/UKB_left_pred.csv'

In [4]:
samples = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/UkBioBank40/interrupted_SC_subjects.csv', header=None)
sample=samples[100:125][0].tolist()

In [7]:
volume=True


bucket_files = []
volume_files = []

for subject_id in sample:

    bck_path = f'{bucket_path}/{subject_id}_cropped_skeleton.bck'
    volume_path = f"{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz"
    
    if volume:
        if os.path.isfile(volume_path):
            vol = aims.read(volume_path)
            volume_files.append(vol)
        else:
            print(f"{volume_path} is not a correct path, or the .nii.gz doesn't exist")
    else:
        if os.path. isfile(bck_path):
            bucket_files.append(bck_path)
        else:
            print(f"{bck_path} is not a correct path, or the .bck doesn't exist")

block = a.createWindowsBlock(5) # 10 columns
dic_windows = {}

if volume:
    for i, vol in enumerate(volume_files):
        dic_windows[f'a_vol{i}'] = a.toAObject(vol)
        #dic_windows[f'a_vol{i}'].setPalette(absoluteMode=True)
        dic_windows[f'rvol{i}'] = a.fusionObjects(objects=[dic_windows[f'a_vol{i}']], method='VolumeRenderingFusionMethod')
        dic_windows[f'rvol{i}'].releaseAppRef()
        dic_windows[f'wvr{i}'] = a.createWindow('3D', block=block)
        dic_windows[f'wvr{i}'].addObjects(dic_windows[f'rvol{i}'])

else:
    for i, file in enumerate(bucket_files):
        dic_windows[f'bck_{i}'] = a.loadObject(file)
        dic_windows[f'w_{i}'] = a.createWindow('3D', block=block)#geometry=[100+400*(i%3), 100+440*(i//3), 400, 400])
        dic_windows[f'w_{i}'].addObjects(dic_windows[f'bck_{i}'])

#### To visualize the white matter for specific people 

In [9]:
'''
list_subjects = ['sub-4298822',
 'sub-5486947',
 'sub-5786451',
 'sub-2703317',
 'sub-4018527',
 'sub-4713688',
 'sub-1641251',
 'sub-4157001',
 'sub-1862053',
 'sub-3892280']

'''
list_subjects = ['sub-5661598',
 'sub-1611989',
 'sub-1696980',
 'sub-5180874',
 'sub-4684085',
 'sub-3051292',
 'sub-4295302',
 'sub-4716793',
 'sub-4868612',
 'sub-4218716']


block2 = a.createWindowsBlock(5) # 5 columns
dic_windows2 = {}

for subject in list_subjects:
    path_to_t1mri = f'/volatile/ad279118/Imaging_Genetics_2025/Figures_Report/TGCC_UKB/{subject}/ses-2/anat/t1mri/default_acquisition'
    dic_windows2[f'w{subject}'] = a.createWindow("3D", block=block2)

    dic_windows2[f'white_{subject}'] = a.loadObject(f'{path_to_t1mri}/default_analysis/segmentation/mesh/{subject}_{side}white.gii')
    dic_windows2[f'white_{subject}'].loadReferentialFromHeader()

    dic_windows2[f'sulci_{subject}'] = a.loadObject(f'{path_to_t1mri}/default_analysis/folds/3.1/{side}{subject}.arg')
    dic_windows2[f'sulci_{subject}'].loadReferentialFromHeader()

    dic_windows2[f'w{subject}'].addObjects([dic_windows2[f'white_{subject}'], dic_windows2[f'sulci_{subject}']])

nifti transfo: 2
memory limit: 43795136512
Reading FGraph version 3.1


bounding box found : 94, 22, 25
                     175, 206, 146
nifti transfo: 3
nifti transfo: 1
memory limit: 43799219404
Reading FGraph version 3.1


bounding box found : 75, 34, 40
                     151, 205, 158
nifti transfo: 2
nifti transfo: 1
memory limit: 43824624435
Reading FGraph version 3.1


bounding box found : 80, 30, 45
                     150, 202, 152
nifti transfo: 2
nifti transfo: 2
memory limit: 43812162764
Reading FGraph version 3.1


bounding box found : 84, 26, 42
                     159, 197, 163
nifti transfo: 3
nifti transfo: 2
memory limit: 43807879987
Reading FGraph version 3.1


...

bounding box found : 79, 35, 52
                     151, 201, 157
nifti transfo: 3
nifti transfo: 1
memory limit: 43829329920
Reading FGraph version 3.1


bounding box found : 70, 32, 39
                     139, 197, 147
nifti transfo: 2
memory limit: 43812021862
Reading FGraph version 3.1


bounding box found : 82, 24, 35
                     157, 198, 149
nifti transfo: 1


Position : 93.1992, 84.5088, 94.8448, 0


: 

# load views as np then plot nifti

In [8]:
#views_dir = '/neurospin/dico/jlaval/Output/2025-03-04/14-55-02_27/logs/views/view1.npy'
#views_dir = '/neurospin/dico/jlaval/Output/2025-03-24/11-38-14_155/logs/views/view1.npy'
views_dir = '/neurospin/dico/jlaval/Output/2025-04-07/13-13-37_123/logs/views/view1.npy'
arr_views = np.load(views_dir)
arr_views.dtype

FileNotFoundError: [Errno 2] No such file or directory: '/neurospin/dico/Output/2025-04-07/13-13-37_123/logs/views/view1.npy'

In [3]:
arr_views.shape

(32, 1, 30, 38, 22)

In [4]:
np.sum(arr_views)

14478.0

In [5]:
volume_files=[]
for view in arr_views[:16]:
    view = np.transpose(view, (1,2,3,0))
    vol = aims.Volume(view)
    volume_files.append(vol)

In [6]:
vol.shape

(30, 38, 22, 1)

In [7]:
block = a.createWindowsBlock(4) # 10 columns
dic_windows = {}

for i, vol in enumerate(volume_files):
    dic_windows[f'a_vol{i}'] = a.toAObject(vol)
    #dic_windows[f'a_vol{i}'].setPalette(absoluteMode=True)
    dic_windows[f'rvol{i}'] = a.fusionObjects(objects=[dic_windows[f'a_vol{i}']], method='VolumeRenderingFusionMethod')
    dic_windows[f'rvol{i}'].releaseAppRef()
    dic_windows[f'wvr{i}'] = a.createWindow('3D', block=block)
    dic_windows[f'wvr{i}'].addObjects(dic_windows[f'rvol{i}'])